
# Quick Start

**dSpace provides simple, universal APIs to program and compose abstractions in smart spaces.**

Define a digi schema:
```yaml
group: digi.dev
version: v1
kind: Plug
control:
  power: string
``` 

Program the digi's driver:
```python
from digi import on

import pytuya
plug = pytuya.Plug("DEVICE_ID")

@on.control("power")
def h(power):
    plug.set(power["intent"])
```

Build and Run:
```bash
dq build plug; dq run plug plut-test
```

That's it! You can now specify the digi's (plug-test) intent in its model:
```yaml
apiVersion: digi.dev/v1
kind: Plug        
metadata:
  name: plug-test
spec:
  control:        
    power:
      # 'I want the plug switched off'
      intent: "off"
```

Apply the model via kubectl:
`kubectl apply -f MODEL_YAML`

We describe some useful concepts about dSpace in what follows.

## dSpace Concepts

**Digi** The basic building block in dSpace is called _digi_. Each digi has a _model_ and a _driver_. A model consists of attribute-value pairs organized in a document (JSON) following a predefined _schema_. The definition of a digi model describing what attributes and the value's data types are used to describe the digi. The fields `sroup`, `version`, and `kind` are the identifier of the schema, the `control.power` is the attribute describes the plug's power states, and `string` is the data type. 

The goal of the digivice driver is to take actions bringing the current states to desired states.Developers can then write the plug's driver and deploy the digi on dSpace. We will describe driver programming in more details later. Once the digi is running, one can interact with it via the model, e.g., declare its desired states of the plug as below:

At development time, developers define the schema and program the driver. At run-time, users declare the desired states of the digi in its model and the driver will fulfill/reconcile the desired states and the actual states. Digis are _composable_; when so, these digis' models will be synced and updated _reflecting how they are composed_.

In this tutorial, we will focus on a special type of digi called _digivice_ (e.g., the Plug in Quick Start). A digivice model has control attributes (e.g., `control.power` in Plug) where each control attribute has an intent field (tracking the desired states) and a status field (tracking the current states). 

Digivices can be composed via the _mount_ operator, forming digivice hierarchies where intents and status flow between digivices. Mounting a digi A to another digi B will allow B's intent to flow to A. For convenience, we refer to A as the child and B as the parent.

> Note: there are format differences between the schema yaml and the model yaml. The latter preserves the [Kubernetes resource model](https://kubernetes.io/docs/concepts/overview/working-with-objects/kubernetes-objects/) as we will be using its CLI to update digis. One can declare the schema yaml in the slightly more verbose k8s-native format; or the model yaml in the dSpace-format likewise.

> Note: if a digivice does not mount any other digis, we refer to it as the "leaf" digivice; otherwise we refer to it as a "higher-layer" digivice. The former are typically the ones that interacts directly to physical device(s).

## How to use this notebook

To run a notebook cell, you can click the Run button in the panel or hit `shift + ENTER` as a shortcut. After you do so, any outputs will appear on the cell's output section. 

> Note: the notebook contains a few macros (e.g., `%elapsed_time`, `%%writefile`); you should be able to safely ignore them. Commands with a leading macro `!` are ones will be executed in shell.

# Example: Home Space

In this tutorial, we will learn how to implement a simple declarative space for home in dSpace. This example space includes lamps, rooms, motion sensors, and the home. We will be able to compose these abstractions and define policies that actuate/automate them. 

As a simple **programming assignment**, we left a few lines of code/configuration for you to fill to help strengthen understanding. They are marked by `YOUR CODE HERE`. The tutorial should take about 15-30 min to walk through. 

## Setup and tools

**dq**: dSpace's command line manager. It is used to build a digi-image (`dq build`) and run a digi (`dq run`). 
**kubectl**: Kubernetes's command line tool to update and check digi's states. More examples are included inline.

In [1]:
from tutorial import (
    create,
    model_file,
    handler_file,
)
%elapsed_time

0:00:00


# Simple Lamp digivice

This simple lamp digivice will sets its TBD. In reality ..

## Define a schema

In [ ]:
%%elapsed_time

schema = """
group: digi.dev
version: v1
kind: Lamp
control:
  power: string  
  # Add a brightness attribute to the lamps's 
  # schema. The following data types are allowed: 
  # {number, integer, string, array,object}
  # YOUR CODE HERE
  brightness: number
"""

create(schema)

## Define a model

In [ ]:
m = model_file("lamp")

In [ ]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: {{ .Values.name }}
spec:
  control:        
    power:
      intent: "on"
    brightness:
      intent: 0.8

## Implement a driver

In [ ]:
f = handler_file("lamp")

In [ ]:
%%elapsed_time
%%writefile $f

from digi import on

@on.control("power")
def h(sv):
    sv["status"] = sv["intent"]

@on.control("brightness")
def h(sv):
    # TO-FILL: set the status of brightness
    sv["status"] = sv["intent"]

## Build

In [ ]:
!dq build lamp -q  # quiet
!dq image

## Run

In [ ]:
!dq run lamp lamp-test
# !dq stop lamp lamp-test

## Read status

In [ ]:
!kubectl get lamp.digi.dev lamp-test -oyaml | kubectl neat

## Update intent

One can modify the intent field of control attributes to update the desired states of the digivice. Here, let's update the desired brightness to 0.1 (previously 0.8).

In [ ]:
m = model_file("lamp", new=False)

In [ ]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: lamp-test
spec:
  control:        
    power:
      intent: "on"
    brightness:
      intent: 0.1

In [ ]:
!kubectl apply -f $m 2> /dev/null  
!kubectl get lamp lamp-test -oyaml | kubectl neat

In [ ]:
# Alternatively, one can patch using a string (the previous method is preferred!), e.g.,
!kubectl patch lamp lamp-test -p '{"spec":{"control":{"power":{"intent":"on"}}}}' --type=merge

# HL abstraction: Room digivice

## Implementation

In [ ]:
%%elapsed_time

# define schema
schema = """
group: digi.dev
version: v1
kind: Room
control:
  brightness: number
mount:     # mount reference
  digi.dev/v1/lamps: object
"""

create(schema)

In [ ]:
# specify model
m = model_file("room")

In [ ]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Room        
metadata:
  name: {{ .Values.name }}
spec:
  control:        
    brightness:
      intent: 0.8

In [ ]:
# implement driver
f = handler_file("room")

In [ ]:
%%elapsed_time
%%writefile $f

from digi import on, logger
from digi.view import TypeView, DotView

@on.mount
@on.control
def h(proc_view):    
    with TypeView(proc_view) as tv, DotView(tv) as dv:  
        # logger.info(dv)
        room_brightness = dv.root.control.brightness
        
        # TO-FILL: set the status of the brightness
        room_brightness.status = 0   
        
        if "lamps" not in dv:
            return

        active_lamps = [l for _, l in dv.lamps.items() 
                        if l.control.power.status == "on"]
        for l in active_lamps:
            room_brightness.status += l.control.brightness.status 
            
            # TO-FILL: update lamp's intent
            l.control.brightness.intent = room_brightness.intent / len(active_lamps) 

In [ ]:
# build and run
!dq build room -q
!dq run room room-test

## Debug

In [ ]:
!kubectl get rooms room-test -oyaml | kubectl neat

In [ ]:
!dq log room-test

## Mount lamps

In [ ]:
!dq mount lamp-test room-test

In [ ]:
!kubectl get rooms room-test -oyaml | kubectl neat

In [ ]:
# start a new lamp-test-2
!dq run lamp lamp-test-2

In [ ]:
!dq mount lamp-test-2 room-test

In [ ]:
!kubectl get rooms room-test -oyaml | kubectl neat

Or [watch the changes](http://localhost:8881/notebooks/display.ipynb).

## Playing with room brightness

In [ ]:
!kubectl patch room room-test -p '{"spec":{"control":{"brightness":{"intent":1}}}}' --type=merge

In [ ]:
!kubectl get rooms room-test -oyaml | kubectl neat

## Connect to physical lamps

...

# Activate the Room with motion

## Pull the MotionSensor

In [ ]:
!dq pull motionsensor

In [ ]:
# run the motion sensor
!dq run motionsensor motion-test

In [ ]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

In [ ]:
# update its sensitivity
!kubectl patch motionsensor motion-test -p '{"spec":{"control":{"sensitivity":{"intent":10}}}}' --type=merge

In [ ]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

## Modify Room

In [ ]:
# Update the Room's schema s.t. it can mount 
# the motionsensor and support on-model policies/reflexes

schema = """
group: digi.dev
version: v1
kind: Room
control:
  brightness: number
mount:     
  digi.dev/v1/lamps: object
  mock.digi.dev/v1/motionsensors: object
reflex: object
"""

create(schema, new=False)

In [ ]:
# build and run
!dq build room -q
!dq run room room-test

## Mount a motionsensor

In [ ]:
!dq mount motion-test room-test

In [ ]:
!dq mount lamp-test room-test

In [ ]:
!dq mount lamp-test-2 room-test

In [ ]:
!kubectl get room room-test -oyaml | kubectl neat

## Add Reflex

In [ ]:
# specify model
m = model_file("room", new=False)

In [ ]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
spec:
  control:
    brightness:
      intent: 0.8
  reflex:
    motion-mode:  
      policy: 'if $time - ."motion-test".obs.last_triggered_time <= 600 then .root.control.brightness.intent = 1 else . end'
      priority: 0
      processor: jq

In [ ]:
!kubectl apply -f $m 2> /dev/null  

In [ ]:
!kubectl get room room-test -oyaml | kubectl neat

## Connect to real motion detectors

...

# Post tutorial question:
> Which part did you find most difficult?

> Any suggestions on the programming/tools? 

# Home (Bonus)

## Pull the base image

In [ ]:
!dq pull home

## Modify the Home driver

Goal: Home have a "mode" control attribute that allows one to tune it to predefined modes. Each mode decides the brightness of the Rooms that mounted to the Home.

In [ ]:
f = handler_file("home")

In [ ]:
%%elapsed_time
%%writefile $f

# TBD
import digi
import digi.on as on

# validation
@on.attr
def h():
    ...

# intent back-prop
@on.mount
def h():
    ...

# status
@on.mount
def h():
    ...

# intent
@on.mount
@on.control
def h():
    ...

In [ ]:
# build and run
!dq build home -q
!dq run home home-test

## Mount rooms

In [ ]:
!dq mount room-test home-test

In [ ]:
# set the modes
!kubectl patch home home-test -p '{"spec":{"control":{"mode":{"intent":YOURMODE}}}}' --type=merge